# Introduction to vector and raster data manipulation with `geopandas` and `rasterio`

This section will borrow heavily from the NCEAS/Arctic Data Center [workshop material](https://learning.nceas.ucsb.edu/2022-09-arctic/sections/10-geopandas.html)!

Let's go exploring on the Arctic Data Center's [catalog](https://arcticdata.io/catalog/data)! I searched for `.tif` files and found a dataset from Berner, L.T., P. Jantz, K.D. Tape, and S.J. Goetz. 2018. Tundra plant above-ground biomass and shrub dominance mapped across the North Slope of Alaska. Environmental Research Letters. 13(3):035002. https://doi.org/10.1088/1748-9326/aaaa9a

In [ ]:

import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import box
import numpy as np
import numpy.ma as ma # numpy mask module
import urllib

In [ ]:
# pip install rasterio
import rasterio

# Part 1: Raster data

Below, we are going to pull data directly off of the Arctic Data Center's repository. The URL is for the data entry, which is the first argument in  `urlretrieve`, and the second argument is the specific file to be downloaded

In [ ]:
url = 'https://cn.dataone.org/cn/v2/resolve/urn:uuid:8233336b-0b1c-4809-b01e-4786f26db63a'

msg = urllib.request.urlretrieve(url, "shrub_dominance_of_agb_p50.tif")

Let's take a look at the `.tif` we just downloaded

In [ ]:
fig, ax = plt.subplots(dpi=200)
with rasterio.open("shrub_dominance_of_agb_p50.tif", masked=True) as shrub_dom:
    # read in raster (1st band)
    shrub_arr = shrub_dom.read(1)
    shrub_arr = ma.masked_where(shrub_arr == shrub_dom.nodata, shrub_arr)
    # shrub_arr = shrub_arr
    shrub_meta = shrub_dom.profile

im0 = ax.imshow(shrub_arr)
cb = fig.colorbar(im0, ax=ax, label="shrub dominance (shrub/plant AGB)",  orientation='horizontal', fraction=0.04, pad=0.2)
print(shrub_meta)

What's going on here?
- we instantiated a `fig` and `ax` object
- we used `numpy.ma` to mask out the nodata values (which are 255.0)
- we printed out the metadata for the raster
- we used `imshow()` on the `ax` object to display the values of the array 
- the axes have an origin at the bottom left of (0,0) because we did not provide `imshow` an `extent`, which we will see later, that can put the x and y axes in real space

FWIW I think there is maybe some issue here, I feel like there should be more data than the nodata mask implies...whatever, go check out the [original work](https://iopscience.iop.org/article/10.1088/1748-9326/aaaa9a)

# Part 2: vector data

Next let's look at vector data. I have already downloaded for you the Circum-Arctic permafrost extent map from the NSIDC [here](https://nsidc.org/data/ggd318/versions/2)

In [ ]:
permafrost = gpd.read_file("../arctic-data/permaice.shp")

permafrost.plot()

Let's transform this to the [coordinate reference system](https://geopandas.org/en/stable/docs/user_guide/projections.html) that our raster is in 

In [ ]:
permafrost = permafrost.to_crs("EPSG:3338")

permafrost.plot()

You'll see that geopandas is pretty good at reprojecting, but man are conical polar projections tricky math! If you go right to pseudo-Mercator from a polar projection, you'll smear these shapefiles across the Arctic. Instead, let's clip the permafrost shapefile to the extent of our shrub dominance raster

In [ ]:
with rasterio.open("shrub_dominance_of_agb_p50.tif", masked=True) as shrub_dom:
    # Use Shapely to make a box geodataframe object from the bounds of the raster
    bbox = box(shrub_dom.bounds[0], shrub_dom.bounds[1], shrub_dom.bounds[2], shrub_dom.bounds[3])
    coord_box_df = gpd.GeoDataFrame(crs = 'EPSG:3338',geometry = [bbox])

# ...and then clip to it!

permafrost_clip = permafrost.clip(coord_box_df)

permafrost_clip.plot()

OK, but this blue is boring. What is even in this object anyway?

In [ ]:
permafrost_clip.head()

That's right, geopandas GeoDataFrames are just dataframes with some spatial info! We can look at the columns to see attributes for the shapefiles. "COMBO" is the three letter combination of permafrost extent, ice content, and overburden (soil) thickness. Read the user manual [here](https://nsidc.org/sites/default/files/ggd318-v002-userguide.pdf)

In [ ]:
permafrost_clip.plot(column="COMBO", legend=True)

# Part 3: together!

`rasterio.plot.show(src)` can plot raster data. But rasters are just grids (or arrays) of numbers. If you tell `plt.imshow()` the extent (boundaries) of the image (the data array), you can transform your raster data however you want, and you can plot it with vector data as well. 

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
with rasterio.open("shrub_dominance_of_agb_p50.tif", masked=True) as shrub_dom:
    shrub_arr = shrub_dom.read(1)
    shrub_arr = ma.masked_where(shrub_arr == shrub_dom.nodata, shrub_arr)
    # Don't you love how "extent" coordinates need to be in a different order? :upside-down smiley face:
    extent=[shrub_dom.bounds[0], shrub_dom.bounds[2], shrub_dom.bounds[1], shrub_dom.bounds[3]]
    im0 = ax.imshow(shrub_arr,
    extent=extent,
    # vmin=0.0,
    # vmax=1.0,
    cmap="binary_r",
    zorder=0,
    )

    shape = permafrost_clip.plot(column="COMBO", legend=True, alpha=0.5, zorder=2, ax=ax)

cb = fig.colorbar(im0, ax=ax, label="shrub dominance (shrub/plant AGB)",  orientation='horizontal', fraction=0.04, pad=0.1)

fig.tight_layout()
# Note I am being a bad data visualizer because white is both 100 and nodata. 

# Part 4: Split-apply-combine raster data with vector data

What if we wanted to analyze raster data grouped by vector data properties? The strategy we have here is to [`rasterize`](https://rasterio.readthedocs.io/en/latest/api/rasterio.features.html#rasterio.features.rasterize) our vector data - instead of having polygons with certain properties, we're going to create arrays where there is one value "under" the polygon of interest and another value (0, or null) everywhere else

First we get the necessary data and metadata from our raster of interest

In [ ]:

from rasterio import features

with rasterio.open("shrub_dominance_of_agb_p50.tif", masked=True) as shrub_dom:

    shrub_meta = shrub_dom.profile
    shrub_arr = shrub_dom.read(1)
    shrub_arr = ma.masked_where(shrub_arr == shrub_dom.nodata, shrub_arr)


Now is a great time to [`dissolve`](https://geopandas.org/en/stable/docs/user_guide/aggregation_with_dissolve.html) our polygons based on their `COMBO` values so that we are rasterizing all the values we are interested in at once

In [ ]:
permafrost_clip = permafrost_clip.dissolve(by='COMBO').reset_index()
# resetting the index helps us iterate through the geometries in the next step, though note each NUM_CODE is unique to the COMBO
permafrost_clip

Now we're going to instantiate two [dictionaries](https://docs.python.org/3/tutorial/datastructures.html) to store the resulting data arrays. Joanmarie will elaborate in the workshop but basically you don't want to store lists or arrays in pandas dataframes. Your goal is to store big raw datasets in dictionaries, reduce them, and then stick them back into your dataframe when all is said and done. 

In [ ]:
means_dict = {}
raw_values_dict = {}

And now we shall (1) rasterize our polygons, (2) make a mask over the raster where the polygon value is present, and both (3) store that raw masked array and (4) calculate the mean of that masked array

In [ ]:
for geom, idx in zip(permafrost_clip['geometry'], permafrost_clip.index):
    # I spent WAY too much time messing around with this part, you apparently can't just
    # give rasterize a polygon, it has to either be a MutliPolygon or a list of geometries
    # This code does the latter, and the Arctic Data Center tutorial's only works because their
    # example vector data is accidentally multipolygons, but our data has a polygon
    geom = shape(geom)
    geoms = [(geom, 1)]
    # Now this looks like the ADC example
    rasterized = features.rasterize(
                                    geoms,
                                    out_shape=shrub_arr.shape, # Look like the source raster
                                    transform=shrub_meta['transform'], # Have the geometry of the source raster
                                    all_touched=True # all pixels touched by geometries (as opposed to pixel centers)
                                    )
    # Theoretically instead of individually rasterizing each polygon type
    # You could rasterize the whole thing and instead of 0s and 1s you can
    # make the raster value the index and then do basic array-style analyses
    # Maybe I can offer treats to someone who writes that for me...
    r_index = np.where(rasterized == 1) # Make the mask
    # r_index.filled(np.nan)
    raw_values_dict[idx] = shrub_arr[r_index].compressed() # store all non-masked data (the compressed thing)
    means_dict[idx] = np.nanmean(shrub_arr[r_index].compressed()) # calculate the mean of the data

Now we have a dictionary whose keys are the index and the values are the means of the masked arrays

In [ ]:
means_dict

*Whenever I say "masked array" I always think of:*

![masquerade](https://media.tenor.com/FTk5tv3gF9MAAAAC/musical-phantom-of-the-opera.gif "masquerade")


But I also made a dictionary that stored *all* of the values:

In [ ]:
raw_values_dict

Maybe you want to do further analyses on the data that you don't want to reduce in that loop. For example, maybe you want to set a threshold number of pixels before you allow for a mean to be counted? This is relevant in this example where the polygon for "ocean" has some data in it.

In [ ]:
# len is length of the array, or number of unmasked pixels 
{key: len(value) for key, value in raw_values_dict.items()}

Now we just create a data frame from that dictionary, and join it to the vector data using `pandas` operations.

In [ ]:
import pandas as pd
# create a data frame from the result
means_df = pd.DataFrame.from_dict(means_dict,
                                     orient='index',
                                     columns=['mean_shrub'])

In [ ]:
means_df

[`pd.DataFrame.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) is a common and powerful way to join two dataframes that share common columns or indices (the latter is true in our case)

In [ ]:
perma_data = permafrost_clip.merge(means_df,
                                    left_index=True,
                                    right_index=True,
                                    how='inner')

perma_data

Now make a map! I took some inspiration for treating missing data from the [GeoPandas docs](https://geopandas.org/en/stable/docs/user_guide/mapping.html):

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
res = perma_data.plot(column="mean_shrub",
                        legend=True,
                        legend_kwds={'label': "Shrub dominance (shrub/plant AGB)",
                        'orientation': "horizontal"},
                        missing_kwds={
                        "color": "lightgrey",
                        "edgecolor": "red",
                        "hatch": "///",
                        "label": "No values",
                        },
                        ax=ax
                        )

So yeah, maybe you'd write a lil script that turns data into NaNs if the threshold is below a certain number...or you just decide to go in and get rid of the data in the ocean. :shrug:
Of course this analysis is a little meaningless as the polygons are huge, but you can see how this would work!

Another option is to just install and use [`rasterstats`](https://github.com/perrygeo/python-rasterstats/tree/d188eaf1f1c20c3ef33aad407f55f9fce51a1220), whose source code I have stolen to write the above snippets

# A note: point data

So, you can use `rasterio` and `geopandas` to [sample](https://rasterio.readthedocs.io/en/latest/api/rasterio.sample.html) point data. A good example can be found in the [GeoPandas docs](https://geopandas.org/en/stable/gallery/geopandas_rasterio_sample.html). If you want to sample an area around a point, consider employing a [buffer](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.buffer.html).

# Hackathon prompt:

Between the Arctic Data Center, the [National Snow and Ice Data Center](https://nsidc.org/home), the [Arctic Permafrost Geospatial Centre](https://apgc.awi.de/), and others, there are so many vector and raster datasets to cross-analyze! Go do some science. 